In [269]:
import numpy as np
import pandas as pd
from scipy.stats import poisson, uniform, norm, expon
import datetime as dt
from dataclasses import dataclass, field
import queue, threading
import random

In [3]:
# coffee chain simulation
# a queue makes sure that a job is run sequentially
# simulation is a great way to determine how each customer is waiting.
# inventory management
# menu popularity

In [642]:
# read hypothetical menu
# all prices are hypothetical

menu = pd.read_csv("starbucks-menu-nutrition-drinks.csv")

In [647]:
# get customer arrival for the day

def get_timestamp(start_time,mu= 3,size=100):
    """
    start_time : datetime_object
    mu : mu parameter for exponential distribution
    
    """
    arrival_secs = np.cumsum(expon.rvs(loc = 1/mu , size= size))
    timestamps = [start_time+dt.timedelta(minutes=x) for x in arrival_secs]
    str_time = [timestamps for x in timestamps]
    return timestamps

# 200 customer arrivals
arrival_times = get_timestamp(start_time, size=200)

In [649]:
class Customer:
    def __init__(self,arrival_time):
        self.arrival_time = arrival_time
    

    def get_order(self, menu):
        idx = random.choices(menu.index, weights=menu["weight"])
        return menu.loc[idx].to_dict(orient="records")[0]
    
    
    
class Barista:
    def __init__(self,name,score,menu):
        self.name = name
        self.score = score
        self.menu = menu
        self.processing_time = dt.timedelta(minutes=0)
        self.wait_time = dt.timedelta(minutes=0)
        self.latest_served_at = dt.datetime(1970,1,1)
        
    def assign(self, customer):
        order_data = {}
        order = customer.get_order(self.menu)
        order_size = random.choices(["small","medium","large"])[0]
        arrival_time = customer.arrival_time
        processing_time = dt.timedelta(minutes = order["processing_time"])
        
        # if customer arrived earlier than latest order finish time
        if self.latest_served_at > arrival_time:
            # customer has to wait until the earlier order is ready
            self.wait_time = self.latest_served_at - arrival_time
        else:
            self.wait_time = dt.timedelta(minutes=0)
            
        order_begin_time = arrival_time + self.wait_time
        order_end_time = order_begin_time + processing_time
        self.latest_served_at = order_end_time
            
        
        # update current processing time
        self.processing_time = dt.timedelta(minutes=order["processing_time"])
        

        order_price = order.get(order_size)
        order_data["barista_name"] = self.name
        order_data["arrival_time"] = str(arrival_time)
        order_data["begin_time"] = str(order_begin_time)
        order_data["processing_time"] = self.processing_time.seconds
        order_data["end_time"] = str(order_end_time)
        order_data["wait_time"] = self.wait_time.seconds
        order_data["item_name"] = order["Name"]
        order_data["order_size"] = order_size
        order_data["order_price"] = order_price
        
        return order_data
        
              
# the more barita a store employes
class Store:
    
    """
    Orchestrates overall activity of the coffee store
    """
    def __init__(self,baristas):
        self.baristas = baristas
        self.reset_barista_availability()
        
        
    def reset_barista_availability(self):
        for x in self.baristas:
            x.processing_time=0
        
    
    def next_barista(self,customer):
        processing_times = [(customer.arrival_time - x.latest_served_at).seconds for x in self.baristas]
        next_barista = self.baristas[np.argmin(processing_times)]
        return next_barista
    
    def get_inventory(self):
        # as customers order an item, inventory runs out
        pass
    

class Simulator:
    def __init__(self, begin_date):
        self.begin_date = begin_date
        
    

    

barista1 = Barista("Dave",90,menu)
barista2 = Barista("Lucy",100,menu)
barista3 = Barista("Ella",100,menu)


baristas = [barista1, barista2]
customer = Customer(arrival_times[0])
store = Store(baristas)

In [652]:
# queue system 
q = queue.Queue()

output = []

def worker():
    
    store = Store(baristas)
    while True:
        
        arrival = q.get()
        customer = Customer(arrival)
        order_data = store.next_barista(customer).assign(customer)
        
        # print(f"working on {item}")
        
        output.append(order_data)
        
        # print(f"order data {order_data}")
        q.task_done()
    else:
        print("Process Finished")

# Thread will stop when store hours are done
threading.Thread(target=worker, daemon=True).start()
        
for item in arrival_times:
    q.put(item)
    

In [651]:
pd.DataFrame(output)

,barista_name,arrival_time,begin_time,processing_time,end_time,wait_time,item_name,order_size,order_price
0,Dave,2020-01-01 09:02:13.343614,2020-01-01 09:02:13.343614,83,2020-01-01 09:03:36.929381,0,Iced White Chocolate Mocha,large,4.703607
1,Lucy,2020-01-01 09:03:04.582744,2020-01-01 09:03:04.582744,78,2020-01-01 09:04:22.905540,0,Iced Caramel Macchiato,large,5.317238
2,Lucy,2020-01-01 09:04:23.632223,2020-01-01 09:04:23.632223,74,2020-01-01 09:05:37.965529,0,Latte Macchiato,medium,4.258168
3,Lucy,2020-01-01 09:06:55.956373,2020-01-01 09:06:55.956373,64,2020-01-01 09:08:00.002716,0,Vanilla Latte,medium,2.930000
4,Lucy,2020-01-01 09:08:03.658541,2020-01-01 09:08:03.658541,77,2020-01-01 09:09:20.781175,0,Iced Skinny Cinnamon Dolce Latte,small,2.490000
...,...,...,...,...,...,...,...,...,...
195,Dave,2020-01-01 13:18:48.181852,2020-01-01 13:57:38.718261,218,2020-01-01 14:01:17.436388,2330,Blonde Roast,medium,4.552441
196,Lucy,2020-01-01 13:19:41.088182,2020-01-01 13:19:41.088182,208,2020-01-01 13:23:09.730430,0,Caffè Latte,medium,4.520431
197,Dave,2020-01-01 13:20:21.827915,2020-01-01 14:01:17.436388,218,2020-01-01 14:04:56.154515,2455,Blonde Roast,small,4.112441
198,Dave,2020-01-01 13:21:17.393897,2020-01-01 14:04:56.154515,212,2020-01-01 14:08:28.716680,2618,Espresso Con Panna,small,4.606821
